# Environment Set Up

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U pip
!pip install transformers==4.36.2
!pip install emoji
!pip install scikit-learn
!pip install nltk
!pip install emoji
!pip install nltk emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 100.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.3
    Uninstalling transformers-4.50.3:
      Successfully uninstalled transformers-4.50.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 w

In [3]:
!pip install datasets

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_m

# Report Overview: Sentiment Classification with BERTweet

This report explores the sentiment classification component of the **BERTweet** paper: [BERTweet Paper](https://arxiv.org/abs/2005.10200). Below is an outline of the work undertaken:

- **Reproducing Sentiment Classification from the Paper**:  
  I reproduced the sentiment classification task from the paper using the **SemEval-2017 Task 4** dataset: [SemEval-2017 Task 4](https://alt.qcri.org/semeval2017/task4/), achieving an **F1-score of 71%**, which is close to the **78.2%** reported in the paper.

- **Transfer Learning from Twitter to Reddit**:  
  I performed transfer learning by adapting the original Twitter-based BERTweet model to work with **Reddit** data. After fine-tuning the model on Reddit, it delivered strong results, with a **macro average F1-score of 0.93**.

- **Comparing Reproduced and Transfer Learning Models on New Reddit Data**:  
  I compared the reproduced model to the **transfer learning model** (`model_reddit`) on the new Reddit dataset available at [The Reddit Dataset](https://huggingface.co/datasets/SocialGrep/the-reddit-dataset-dataset).  
  The transfer learning model (`model_reddit`) showed significant improvements in **accuracy** and various **F1-scores** compared to the reproduced model.


# 1. Reproduce
This part reproduced the **sentiment classification component** from the BERTweet paper: *"BERTweet: A pre-trained language model for English Tweets"* (Nguyen et al., 2020). The objective was to replicate the fine-tuning of the BERTweet model on SemEval-2017 Task 4A and evaluate its performance on sentiment classification using English tweets.

## 1.1 Load and preprocess Data

- This part preprocessed Dataset **SemEval-2017** using a custom normalization function that mimics the TweetNormalizer module used in the BERTweet paper.

- Detect twitter-2016test-A-English.txt (SemEval always collect the data of last whole year), we nooticed that the Dataset is **not balance**, so I downsampled neutral and positive, to make the data to **negative: neutral: positive = 3000: 3000: 3000**

- To evaluate the model's generalization, we randomly split the data into training and validation sets using an 80/20 ratio with stratification to preserve class balance.



In [4]:
# Load Data
label_path = "/content/drive/My Drive/Colab Notebooks/BetTweet/4A-English/twitter-2016test-A-English.txt"
with open(label_path, "r", encoding="utf-8") as f:
    raw_labels = [line.strip().split('\t')[1] for line in f if '\t' in line]

# Lable Number
from collections import Counter
label_counts = Counter(raw_labels)
for label, count in label_counts.items():
    print(f"{label}: {count}")

neutral: 10342
negative: 3231
positive: 7059


In [5]:
from nltk.tokenize import TweetTokenizer
from emoji import demojize
import pandas as pd
from collections import Counter
from sklearn.utils import resample

# Initialize the tweet tokenizer
tokenizer = TweetTokenizer()

# Normalize individual tokens
def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

# Normalize an entire tweet
def normalizeTweet(tweet):
    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])
    normTweet = (
        normTweet.replace("cannot ", "can not ")
        .replace("n't ", " n't ")
        .replace("n 't ", " n't ")
        .replace("ca n't", "can't")
        .replace("ai n't", "ain't")
    )
    normTweet = (
        normTweet.replace("'m ", " 'm ")
        .replace("'re ", " 're ")
        .replace("'s ", " 's ")
        .replace("'ll ", " 'll ")
        .replace("'d ", " 'd ")
        .replace("'ve ", " 've ")
    )
    normTweet = (
        normTweet.replace(" p . m .", "  p.m.")
        .replace(" p . m ", " p.m ")
        .replace(" a . m .", " a.m.")
        .replace(" a . m ", " a.m ")
    )
    return " ".join(normTweet.split())

# Load and normalize tweets
with open("/content/drive/My Drive/Colab Notebooks/BetTweet/4A-English/SemEval2017-task4-dev.subtask-A.english.INPUT.txt", "r", encoding="utf-8") as f:
    raw_tweets = [line.strip() for line in f.readlines()]
tweets = [normalizeTweet(t) for t in raw_tweets]
print(f"Loaded and normalized {len(tweets)} tweets.")

# Load tweet labels
label_path = "/content/drive/My Drive/Colab Notebooks/BetTweet/4A-English/twitter-2016test-A-English.txt"
with open(label_path, "r", encoding="utf-8") as f:
    raw_labels = [line.strip().split('\t')[1] for line in f if '\t' in line]

label2id = {"negative": 0, "neutral": 1, "positive": 2}
labels = [label2id[l] for l in raw_labels]

# Trim to equal length
min_len = min(len(tweets), len(labels))
tweets = tweets[:min_len]
labels = labels[:min_len]

# Zip into one dataset
train_data = list(zip(tweets, labels))

# Separate data by label
neutral_data = [x for x in train_data if x[1] == 1]
positive_data = [x for x in train_data if x[1] == 2]
negative_data = [x for x in train_data if x[1] == 0]

# Target: keep negative at original 3231, make others match it
target_samples = 3231

# Downsample neutral to 3231
neutral_data_balanced = resample(neutral_data,
                                 replace=False,
                                 n_samples=target_samples,
                                 random_state=42)

# Upsample positive to 3231
positive_data_balanced = resample(positive_data,
                                  replace=True,
                                  n_samples=target_samples,
                                  random_state=42)

# Keep negative unchanged (assumed to already be 3231)
negative_data_balanced = negative_data  # No change

# Combine all classes into one balanced dataset
balanced_train_data = neutral_data_balanced + positive_data_balanced + negative_data_balanced

# Extract tweet texts and labels
tweets_train_balanced = [x[0] for x in balanced_train_data]
labels_train_balanced = [x[1] for x in balanced_train_data]

# Display new label distribution
print("Balanced label distribution:", Counter(labels_train_balanced))

Loaded and normalized 20633 tweets.
Balanced label distribution: Counter({1: 3231, 2: 3231, 0: 3231})


## 1.2 Run BERTweet prediction and score
- **Model:**  `vinai/bertweet-base`
- **Tokenizer:** `AutoTokenizer` with `use_fast=False` was employed to ensure compatibility with tweet-style tokenization.
- **Training:** Fine-tuning was performed for 3 epochs with a batch size of 16 and a learning rate of 1e-5.
- **Loss Function:** CrossEntropyLoss with default settings.
- **Hardware:** Google Colab with A100

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch import nn
from sklearn.metrics import classification_report
from collections import Counter

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare Dataset and Dataloader
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(label, dtype=torch.long)
        }

train_dataset = TweetDataset(tweets_train_balanced, labels_train_balanced, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Compute Class Weights
class_counts = Counter(labels_train_balanced)
class_weights = [len(labels_train_balanced) / class_counts[i] for i in range(3)]
class_weights = torch.tensor(class_weights).to(device)

# Training Loop with Weighted Loss
optimizer = AdamW(model.parameters(), lr=1e-5)
model.train()

for epoch in range(3):
    total_loss = 0
    preds = []
    true_labels = []
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        label = batch['label'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=label)
        loss = nn.CrossEntropyLoss(weight=class_weights)(outputs.logits, label)  # Using weighted loss
        total_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Collect predictions and true labels for classification report
        predictions = torch.argmax(outputs.logits, dim=1)
        preds.extend(predictions.cpu().numpy())
        true_labels.extend(label.cpu().numpy())

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}")

    # After each epoch, print classification report
    print("\n=== Classification Report (Training Set) ===")
    print(classification_report(true_labels, preds, labels=[0, 1, 2], target_names=["negative", "neutral", "positive"]))

# Save model
torch.save(model.state_dict(), "/content/drive/My Drive/Colab Notebooks/BetTweet/model_reproduce.pt")


/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to

config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.91M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 0.8653

=== Classification Report (Training Set) ===
              precision    recall  f1-score   support

    negative       0.71      0.57      0.63      3231
     neutral       0.58      0.70      0.63      3231
    positive       0.66      0.64      0.65      3231

    accuracy                           0.64      9693
   macro avg       0.65      0.64      0.64      9693
weighted avg       0.65      0.64      0.64      9693

Epoch 2, Loss: 0.6612

=== Classification Report (Training Set) ===
              precision    recall  f1-score   support

    negative       0.69      0.72      0.71      3231
     neutral       0.69      0.71      0.70      3231
    positive       0.74      0.68      0.71      3231

    accuracy                           0.71      9693
   macro avg       0.71      0.71      0.71      9693
weighted avg       0.71      0.71      0.71      9693

Epoch 3, Loss: 0.5446

=== Classification Report (Training Set) ===
              precision    recall 

## 1.3 Reproducation Summary

The reproduction achieved an accuracy of **70%**, with a macro-average F1-score of **0.71**. This indicates a good level of performance, especially considering the imbalance in the dataset. The model demonstrated strong recall for the negative class (0.77) but showed room for improvement in the recall for the positive (0.66) and neutral (0.69) classes. These results confirm that BERTweet can be effectively fine-tuned for sentiment classification.

When **compared to the original paper**, which reported an F1-score of **78.2%**, our reproduced result of **71%** shows a close match, especially considering that we used only the provided test data and did not rely on additional external training data. The slight drop in **F1-score** can be attributed to differences in training data size and the inherent class imbalance in the dataset. However, the overall consistency between the two results confirms that the BERTweet model maintains robust performance across different setups.

# 2. Contribution

- conducted transfer learning by fine-tuning the same architecture on a Reddit-based sentiment dataset (https://www.kaggle.com/datasets/cosmos98/twitter-and-reddit-sentimental-analysis-dataset ), get model_reddit
-

## 2.1 Load and Preprocess Reddit Data

### 2.1.1 Load and process Data

In [7]:
import pandas as pd

file_path = '/content/drive/My Drive/Colab Notebooks/BetTweet/Reddit_Data.csv'
df = pd.read_csv(file_path)

print("Dataset shape (rows, columns):", df.shape)
print("\n Data types of each column:")
print(df.dtypes)

print("\n Missing values count:")
print(df.isnull().sum())

print("\n Missing value percentage (%):")
print((df.isnull().sum() / len(df) * 100).round(2))


Dataset shape (rows, columns): (37249, 2)

 Data types of each column:
clean_comment    object
category          int64
dtype: object

 Missing values count:
clean_comment    100
category           0
dtype: int64

 Missing value percentage (%):
clean_comment    0.27
category         0.00
dtype: float64


In [8]:
import pandas as pd

# Drop rows with missing clean_comment
df = df.dropna(subset=['clean_comment'])

category_counts = df['category'].value_counts().sort_index()
print(category_counts)



category
-1     8277
 0    13042
 1    15830
Name: count, dtype: int64


In [9]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from collections import Counter

# Load and clean data
file_path = '/content/drive/My Drive/Colab Notebooks/BetTweet/Reddit_Data.csv'
df = pd.read_csv(file_path).dropna(subset=['clean_comment']).reset_index(drop=True)

def clean_text(text):
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#(\w+)", r"\1", text)
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df['clean_comment'] = df['clean_comment'].apply(clean_text)

# Split
X = df['clean_comment']
y = df['category']

tweets_train, tweets_val, labels_train, labels_val = train_test_split(
    X, y, test_size=0.1, stratify=y, random_state=42
)

# Label remapping: -1 -> 0, 0 -> 1, 1 -> 2
label_map = {-1: 0, 0: 1, 1: 2}
labels_train_balanced = [label_map[l] for l in labels_train.tolist()]
labels_val = [label_map[l] for l in labels_val.tolist()]
tweets_train_balanced = tweets_train.tolist()
tweets_val = tweets_val.tolist()

## 2.2 Run Reddit-BERTweet prediction and score

In [10]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from collections import Counter
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", normalization=True)
model_reddit = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=3)
model_reddit.to(device)

# Dataset
class TweetDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create DataLoader
train_dataset = TweetDataset(tweets_train_balanced, labels_train_balanced)
val_dataset = TweetDataset(tweets_val, labels_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

# Set class weights
total = len(labels_train_balanced)
counts = Counter(labels_train_balanced)
weights = [total / counts[i] for i in range(3)]
weights_tensor = torch.tensor(weights).to(device)

loss_fn = CrossEntropyLoss(weight=weights_tensor)
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training
EPOCHS = 4
for epoch in range(1, EPOCHS + 1):
    print(f"\n===== Epoch {epoch} =====")
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc="training"):
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)

        outputs = model(**inputs)
        loss = loss_fn(outputs.logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"\nTraining Loss: {avg_loss:.4f}")

    # Verification
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Verifying"):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)

            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("\n=== Classification Report (Training Set) ===")
    print(classification_report(all_labels, all_preds, target_names=["negative", "neutral", "positive"]))

    # Save
    torch.save(model.state_dict(), f"/content/bertweet_epoch{epoch}.pt")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



===== Epoch 1 =====


training:   0%|          | 0/1045 [00:00<?, ?it/s]


Training Loss: 0.6666


Verifying:   0%|          | 0/59 [00:00<?, ?it/s]

...


=== Classification Report (Training Set) ===
              precision    recall  f1-score   support

    negative       0.83      0.93      0.88       828
     neutral       0.97      0.94      0.96      1304
    positive       0.95      0.92      0.93      1583

    accuracy                           0.93      3715
   macro avg       0.92      0.93      0.92      3715
weighted avg       0.93      0.93      0.93      3715


===== Epoch 4 =====


training:   0%|          | 0/1045 [00:00<?, ?it/s]


Training Loss: 0.1544


Verifying:   0%|          | 0/59 [00:00<?, ?it/s]


=== Classification Report (Training Set) ===
              precision    recall  f1-score   support

    negative       0.81      0.96      0.88       828
     neutral       0.97      0.97      0.97      1304
    positive       0.98      0.89      0.93      1583

    accuracy                           0.93      3715
   macro avg       0.92      0.94      0.93      3715
weighted avg       0.94      0.93      0.93      3715



Overall, the model demonstrates balanced performance across all three classes, with a macro average F1-score of **0.93**, indicating it generalizes well across categories.

## 2.3 Compare reproduce model to reddit model on new reddit dataset

we have reproduce model and model_reddit.
Now we test this two model on new dataset: https://huggingface.co/datasets/SocialGrep/the-reddit-dataset-dataset to see if the trasfer learning successful.

### 2.3.1 Load and Process New data
We used the HuggingFace-hosted dataset SocialGrep/the-reddit-dataset-dataset, which includes Reddit comments and a sentiment score in the range [-1, 1]. We mapped these scores to categorical sentiment labels using thresholds:

- score < -0.2: negative (0)
- -0.2 <= score <= 0.2: neutral (1)
- score > 0.2: positive (2)

We also cleaned the Reddit comment texts to remove URLs, mentions, hashtags, emojis, and excess whitespace.

In [11]:
from datasets import load_dataset, Dataset
import re

# Load dataset
ds = load_dataset("SocialGrep/the-reddit-dataset-dataset", "comments")
data = ds['train']

# Define preprocessing functions
def map_sentiment(example):
    score = example['sentiment']
    if score < -0.2:
        label = 0  # negative
    elif score > 0.2:
        label = 2  # positive
    else:
        label = 1  # neutral
    example['label'] = label
    return example

def clean_body(example):
    text = example['body']
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#(\w+)", r"\1", text)
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    example['clean_body'] = text
    return example

# Apply cleaning functions
data = data.filter(lambda x: x['body'] is not None and x['sentiment'] is not None)
data = data.map(map_sentiment)
data = data.map(clean_body)

# Optional: convert to pandas for train/val split
df = data.to_pandas()

# Train/val split using sklearn
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    df['clean_body'], df['label'], test_size=0.1, stratify=df['label'], random_state=42
)

tweets_train_balanced = X_train.tolist()
labels_train_balanced = y_train.tolist()
tweets_val = X_val.tolist()
labels_val = y_val.tolist()

README.md:   0%|          | 0.00/2.92k [00:00<?, ?B/s]

the-reddit-dataset-dataset.py:   0%|          | 0.00/7.85k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/9.68M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/54848 [00:00<?, ? examples/s]

Filter:   0%|          | 0/54848 [00:00<?, ? examples/s]

Map:   0%|          | 0/47370 [00:00<?, ? examples/s]

Map:   0%|          | 0/47370 [00:00<?, ? examples/s]

## 2.3.2 Comparison Results

In [12]:
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", normalization=True)

# Custom Dataset
class TweetDataset(Dataset):
    def __init__(self, texts):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)

    def __getitem__(self, idx):
        return {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}

    def __len__(self):
        return len(self.encodings["input_ids"])

val_dataset = TweetDataset(tweets_val)
val_loader = DataLoader(val_dataset, batch_size=64)

# === Function to evaluate model ===
def evaluate_model(model_path, model_name="Model"):
    model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=3)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    all_preds = []

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())

    print(f"\n=== Classification Report for {model_name} ===")
    report = classification_report(
        labels_val, all_preds, labels=[0, 1, 2], target_names=["negative", "neutral", "positive"], output_dict=True
    )
    df_report = pd.DataFrame(report).transpose()
    print(df_report[["precision", "recall", "f1-score"]])
    return all_preds, df_report

# === Evaluate both models ===
preds_model, report_model = evaluate_model(
    "/content/drive/My Drive/Colab Notebooks/BetTweet/model_reproduce.pt", "Twitter model"
)
preds_model_reddit, report_model_reddit = evaluate_model(
    "/content/bertweet_epoch4.pt", "Reddit fine-tuned model"
)

# Optional: Combine and display
comparison = pd.concat(
    [report_model[["precision", "recall", "f1-score"]],
     report_model_reddit[["precision", "recall", "f1-score"]]],
    axis=1,
    keys=["model", "model_reddit"]
)
print("\n=== Combined Comparison ===")
print(comparison)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Classification Report for Twitter model ===
              precision    recall  f1-score
negative       0.357702  0.254174  0.297180
neutral        0.317868  0.653144  0.427623
positive       0.634221  0.306730  0.413485
accuracy       0.408909  0.408909  0.408909
macro avg      0.436597  0.404683  0.379429
weighted avg   0.503984  0.408909  0.404666


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Classification Report for Reddit fine-tuned model ===
              precision    recall  f1-score
negative       0.352201  0.519481  0.419790
neutral        0.656525  0.554429  0.601173
positive       0.780171  0.772711  0.776423
accuracy       0.675744  0.675744  0.675744
macro avg      0.596299  0.615540  0.599129
weighted avg   0.692869  0.675744  0.681126

=== Combined Comparison ===
                 model                     model_reddit                    
             precision    recall  f1-score    precision    recall  f1-score
negative      0.357702  0.254174  0.297180     0.352201  0.519481  0.419790
neutral       0.317868  0.653144  0.427623     0.656525  0.554429  0.601173
positive      0.634221  0.306730  0.413485     0.780171  0.772711  0.776423
accuracy      0.408909  0.408909  0.408909     0.675744  0.675744  0.675744
macro avg     0.436597  0.404683  0.379429     0.596299  0.615540  0.599129
weighted avg  0.503984  0.408909  0.404666     0.692869  0.675744  0.681

Although the original model performed well on Twitter, its accuracy was less than 41% when used directly on Reddit. After transfer learning, model_reddit achieved significant improvements in accuracy and various F1-scores, especially in the "positive" category, indicating that the transfer was successful and effective.